In [208]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/coronavirusdataset/TimeAge.csv
/kaggle/input/coronavirusdataset/Case.csv
/kaggle/input/coronavirusdataset/Policy.csv
/kaggle/input/coronavirusdataset/Region.csv
/kaggle/input/coronavirusdataset/PatientInfo.csv
/kaggle/input/coronavirusdataset/SearchTrend.csv
/kaggle/input/coronavirusdataset/Weather.csv
/kaggle/input/coronavirusdataset/TimeProvince.csv
/kaggle/input/coronavirusdataset/TimeGender.csv
/kaggle/input/coronavirusdataset/SeoulFloating.csv
/kaggle/input/coronavirusdataset/Time.csv


* Between the people died or released from the hospital I take their input features , perform some data engineering and feed the features into a logistic regression classifier and predict whether the person died or released from the hospital all using spark frame work
* the purpose of this notebook is to get familiar with spark sql and data frames preprocessing and machine learning in spark

In [209]:
INPUT_PATH = '../input/coronavirusdataset/PatientInfo.csv'

In [210]:
!pip install pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
from pyspark.sql.functions import col 
from pyspark.sql.functions import coalesce
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import datediff,col
from pyspark.ml.feature import VectorAssembler , StringIndexer , OneHotEncoder , Imputer
from pyspark.sql.types import FloatType , IntegerType , StringType
from pyspark.sql.functions import udf




In [211]:
spark_session = SparkSession.builder.appName('lab1').master('local[*]').getOrCreate()

In [212]:
df_raw = spark_session.read.csv(INPUT_PATH , header = True , inferSchema=True , samplingRatio=0.1)
df_raw.show(5,truncate=False)

+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+-------------------+-------------------+-------------------+-------------+--------+
|patient_id|sex   |age|country|province|city       |infection_case      |infected_by|contact_number|symptom_onset_date |confirmed_date     |released_date      |deceased_date|state   |
+----------+------+---+-------+--------+-----------+--------------------+-----------+--------------+-------------------+-------------------+-------------------+-------------+--------+
|1000000001|male  |50s|Korea  |Seoul   |Gangseo-gu |overseas inflow     |null       |75            |2020-01-22 00:00:00|2020-01-23 00:00:00|2020-02-05 00:00:00|null         |released|
|1000000002|male  |30s|Korea  |Seoul   |Jungnang-gu|overseas inflow     |null       |31            |null               |2020-01-30 00:00:00|2020-03-02 00:00:00|null         |released|
|1000000003|male  |50s|Korea  |Seoul   |Jongno-gu  |contact with patient|2002000

In [213]:
df_raw.printSchema()

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: string (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: long (nullable = true)
 |-- contact_number: integer (nullable = true)
 |-- symptom_onset_date: timestamp (nullable = true)
 |-- confirmed_date: timestamp (nullable = true)
 |-- released_date: timestamp (nullable = true)
 |-- deceased_date: timestamp (nullable = true)
 |-- state: string (nullable = true)



In [214]:
df_raw.summary().show()

+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+-------------------+--------+
|summary|          patient_id|   sex| age|   country|province|          city|      infection_case|         infected_by|     contact_number|   state|
+-------+--------------------+------+----+----------+--------+--------------+--------------------+--------------------+-------------------+--------+
|  count|                5165|  4043|3785|      5165|    5165|          5071|                4246|                1345|                785|    5165|
|   mean|2.8636345618679576E9|  null|null|      null|    null|          null|                null|2.2845944015643125E9| 1273903.6675159235|    null|
| stddev| 2.074210725277473E9|  null|null|      null|    null|          null|                null|1.5265072953383324E9|3.569155830456973E7|    null|
|    min|          1000000001|female|  0s|Bangladesh|   Busan|     Andong-si|Anyang Gunpo Past...|        

# number of survived people

In [215]:
# no nulls  in state column
df_raw.select('state').groupBy('state').agg(fn.count(fn.col('state'))).show()
survived_people = df_raw.select('state').where('state = "released" ').count()
not_survived = df_raw.select('state').where('state <> "released"').count()

print('survived people number is ',survived_people)
print('not survived people number is ',not_survived)

+--------+------------+
|   state|count(state)|
+--------+------------+
|isolated|        2158|
|released|        2929|
|deceased|          78|
+--------+------------+

survived people number is  2929
not survived people number is  2236


# nulls in the columns

In [216]:
for column in df_raw.columns:
  n_nulls = df_raw.select(column) \
                  .where(col(column).isNull()) \
                  .count()
  #df_raw.select(column).where(col(column).isNull()).show()
  print('the number of nulls in the column',column , 'is ',n_nulls)
print('the confirmed date is the date he entered the quarantine \n \
  the released date is the date he exited the hospital \n \
  the deceased date is the date he died (if he died) \n \
  ')

the number of nulls in the column patient_id is  0
the number of nulls in the column sex is  1122
the number of nulls in the column age is  1380
the number of nulls in the column country is  0
the number of nulls in the column province is  0
the number of nulls in the column city is  94
the number of nulls in the column infection_case is  919
the number of nulls in the column infected_by is  3820
the number of nulls in the column contact_number is  4380
the number of nulls in the column symptom_onset_date is  4476
the number of nulls in the column confirmed_date is  3
the number of nulls in the column released_date is  3578
the number of nulls in the column deceased_date is  5099
the number of nulls in the column state is  0
the confirmed date is the date he entered the quarantine 
   the released date is the date he exited the hospital 
   the deceased date is the date he died (if he died) 
   


## understanding the nulls in the date columns

In [217]:
## we need to drop these rows (has no release or desease date)
release_with_no_date = df_raw.select('state' ).where('state = "released" ').where(df_raw['released_date'].isNull() ).count()
deseased_with_no_date = df_raw.select('state' , 'deceased_date').where('state = "deceased" ').where(df_raw['deceased_date'].isNull() ).count()
print(release_with_no_date , 'released with no release date')
print(deseased_with_no_date , 'deseased with no decease date')

1350 released with no release date
12 deseased with no decease date


#### the state is deceased and both the release and deceased are not null (released from the hospital and died in home)

In [218]:
df_to_remove= df_raw.filter( ( (df_raw['state'] == "released") \
                                             & (df_raw['released_date'].isNull()) ) ) 
df_to_remove[['released_date' , 'state']].show(5)

## keeping not the condition (== removing the above condition)
df_final = df_raw.filter(~ ( (df_raw['state'] == "released") \
                                & (df_raw['released_date'].isNull()) ) ) 
df_final[['released_date' , 'state']].count()

+-------------+--------+
|released_date|   state|
+-------------+--------+
|         null|released|
|         null|released|
|         null|released|
|         null|released|
|         null|released|
+-------------+--------+
only showing top 5 rows



3815

In [219]:
released_then_deceased = df_final.select('state' , 'deceased_date')\
            .where('state = "deceased" ')\
            .where((df_final['released_date'].isNotNull()) \
                   & (df_final['deceased_date'].isNotNull() ) )\
            .count()
print(released_then_deceased,' people released then deceased at home')

df_final.select('state' , 'deceased_date' , 'released_date')\
            .where('state = "deceased" ')\
            .where((df_final['released_date'].isNotNull()) \
                   & (df_final['deceased_date'].isNotNull() ) ).show()

2  people released then deceased at home
+--------+-------------------+-------------------+
|   state|      deceased_date|      released_date|
+--------+-------------------+-------------------+
|deceased|2020-04-08 00:00:00|2020-03-30 00:00:00|
|deceased|2020-05-11 00:00:00|2020-05-08 00:00:00|
+--------+-------------------+-------------------+



In [220]:
## people with no decease date
df_final.select('state' , 'deceased_date' , 'released_date')\
            .where('state = "deceased" ')\
            .where(  (df_final['deceased_date'].isNull() ) ).show()

+--------+-------------+-------------------+
|   state|deceased_date|      released_date|
+--------+-------------+-------------------+
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|               null|
|deceased|         null|2020-03-24 00:00:00|
+--------+-------------+-------------------+



# Data Processing

## 1- remove the rows with 'isolated' state

In [221]:
df_processed = df_final.filter( (df_raw['state'] == 'released') | (df_raw['state'] == 'deceased') )
df_processed[['state']].show(5)

+--------+
|   state|
+--------+
|released|
|released|
|released|
|released|
|released|
+--------+
only showing top 5 rows



## 1- making a new column (final date)

In [222]:
df_processed = df_processed.withColumn('final date' , coalesce( df_processed['released_date'] , df_processed['deceased_date'] ).cast(TimestampType()))
df_processed[['released_date' , 'deceased_date' , 'final date']].show()
df_processed.where( (df_processed['deceased_date'].isNull()) & (df_processed['released_date'].isNull() ) ).count()
df_processed.where(df_processed['final date'].isNull()).count()

+-------------------+-------------+-------------------+
|      released_date|deceased_date|         final date|
+-------------------+-------------+-------------------+
|2020-02-05 00:00:00|         null|2020-02-05 00:00:00|
|2020-03-02 00:00:00|         null|2020-03-02 00:00:00|
|2020-02-19 00:00:00|         null|2020-02-19 00:00:00|
|2020-02-15 00:00:00|         null|2020-02-15 00:00:00|
|2020-02-24 00:00:00|         null|2020-02-24 00:00:00|
|2020-02-19 00:00:00|         null|2020-02-19 00:00:00|
|2020-02-10 00:00:00|         null|2020-02-10 00:00:00|
|2020-02-24 00:00:00|         null|2020-02-24 00:00:00|
|2020-02-21 00:00:00|         null|2020-02-21 00:00:00|
|2020-02-29 00:00:00|         null|2020-02-29 00:00:00|
|2020-02-29 00:00:00|         null|2020-02-29 00:00:00|
|2020-02-27 00:00:00|         null|2020-02-27 00:00:00|
|               null|         null|               null|
|2020-03-12 00:00:00|         null|2020-03-12 00:00:00|
|2020-03-11 00:00:00|         null|2020-03-11 00

11

#### dropping the nulls in final date (people has no release date or decease date)

In [223]:
df_processed = df_processed.filter(df_processed['final date'].isNotNull())
df_processed.count()

1646

## 2- making a new column (no_days) indicating the number of days in the hospital

In [224]:
df_processed = df_processed.withColumn('no_days' , datediff(end='final date' , start='confirmed_date' ))
df_processed[[ 'confirmed_date' , 'final date' , 'state' , 'no_days' ]].show(5)

+-------------------+-------------------+--------+-------+
|     confirmed_date|         final date|   state|no_days|
+-------------------+-------------------+--------+-------+
|2020-01-23 00:00:00|2020-02-05 00:00:00|released|     13|
|2020-01-30 00:00:00|2020-03-02 00:00:00|released|     32|
|2020-01-30 00:00:00|2020-02-19 00:00:00|released|     20|
|2020-01-30 00:00:00|2020-02-15 00:00:00|released|     16|
|2020-01-31 00:00:00|2020-02-24 00:00:00|released|     24|
+-------------------+-------------------+--------+-------+
only showing top 5 rows



## 3- adding is male column

In [225]:
df_processed = df_processed.withColumn('is_male' , df_processed['sex'] == 'male')
df_processed[['is_male' , 'sex']].show(5)

+-------+------+
|is_male|   sex|
+-------+------+
|   true|  male|
|   true|  male|
|   true|  male|
|   true|  male|
|  false|female|
+-------+------+
only showing top 5 rows



## 4- adding is_dead column (target variable)

##### using udf (just to try them out in spark)

In [226]:
def add_is_dead(x):
    if x == 'released':
        return False
    else:
        return True
is_dead_udf = udf(add_is_dead)
df_processed = df_processed.withColumn('is_dead' , is_dead_udf(df_processed['state']))
df_processed[['is_dead' , 'state']].show(3)
df_processed.select('is_dead' , 'state').where(df_processed['state'] == 'deceased').show(3)

+-------+--------+
|is_dead|   state|
+-------+--------+
|  false|released|
|  false|released|
|  false|released|
+-------+--------+
only showing top 3 rows

+-------+--------+
|is_dead|   state|
+-------+--------+
|   true|deceased|
|   true|deceased|
|   true|deceased|
+-------+--------+
only showing top 3 rows



## 5- processing the age column

In [227]:
df_processed = df_processed.withColumn('age' , fn.translate('age','s','').cast(IntegerType()) )
df_processed[['age']].show(4)

## casting the is_male column to string to be ready for string indexer
df_processed = df_processed.withColumn('is_male' , df_processed['is_male'].cast(StringType()) )
df_processed.printSchema()

+---+
|age|
+---+
| 50|
| 30|
| 50|
| 20|
+---+
only showing top 4 rows

root
 |-- patient_id: long (nullable = true)
 |-- sex: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- infected_by: long (nullable = true)
 |-- contact_number: integer (nullable = true)
 |-- symptom_onset_date: timestamp (nullable = true)
 |-- confirmed_date: timestamp (nullable = true)
 |-- released_date: timestamp (nullable = true)
 |-- deceased_date: timestamp (nullable = true)
 |-- state: string (nullable = true)
 |-- final date: timestamp (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: string (nullable = true)
 |-- is_dead: string (nullable = true)



## 7- drop the unwanted columns

In [228]:
df_processed = df_processed.drop("patient_id","sex","infected_by","contact_number","released_date","state", "symptom_onset_date","confirmed_date","deceased_date","country","final date", "city","infection_case")
df_processed.show(6)

+---+--------+-------+-------+-------+
|age|province|no_days|is_male|is_dead|
+---+--------+-------+-------+-------+
| 50|   Seoul|     13|   true|  false|
| 30|   Seoul|     32|   true|  false|
| 50|   Seoul|     20|   true|  false|
| 20|   Seoul|     16|   true|  false|
| 20|   Seoul|     24|  false|  false|
| 50|   Seoul|     19|  false|  false|
+---+--------+-------+-------+-------+
only showing top 6 rows



# the machine learning model

In [229]:
df_final = df_processed.alias('df_final')

In [230]:
id(df_final) == id(df_processed)

False

In [231]:
df_final.printSchema()

root
 |-- age: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- no_days: integer (nullable = true)
 |-- is_male: string (nullable = true)
 |-- is_dead: string (nullable = true)



Sanity check on nulls and drop them

In [232]:
print([(str(col) ,df_final.select(col).where(fn.col(col).isNull()).count() ) for col in df_final.columns])
df_final.select('age').where(df_final['age'].isNull()).show(5)
df_final.select('is_male').where(df_final['is_male'].isNull()).show()

## only 12 null rows we can drop them easily
df_final = df_final.filter(df_final['age'].isNotNull()).filter(df_final['is_male'].isNotNull())
df_final.show(5)
print([(str(col) ,df_final.select(col).where(fn.col(col).isNull()).count() ) for col in df_final.columns])


[('age', 11), ('province', 0), ('no_days', 0), ('is_male', 1), ('is_dead', 0)]
+----+
| age|
+----+
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 5 rows

+-------+
|is_male|
+-------+
|   null|
+-------+

+---+--------+-------+-------+-------+
|age|province|no_days|is_male|is_dead|
+---+--------+-------+-------+-------+
| 50|   Seoul|     13|   true|  false|
| 30|   Seoul|     32|   true|  false|
| 50|   Seoul|     20|   true|  false|
| 20|   Seoul|     16|   true|  false|
| 20|   Seoul|     24|  false|  false|
+---+--------+-------+-------+-------+
only showing top 5 rows

[('age', 0), ('province', 0), ('no_days', 0), ('is_male', 0), ('is_dead', 0)]


### replacing the invalid ages (0 or less) by the median of ages

In [233]:
print(df_final.select('*').where(df_final['age'] <= 0).count() )
age_imputer = Imputer(inputCol = 'age' , outputCol='age_imputed')
df_final = age_imputer.setStrategy('median').setMissingValue(0).fit(df_final).transform(df_final)
df_final.select('age', 'age_imputed').where(df_final['age']==0).show(5)


18
+---+-----------+
|age|age_imputed|
+---+-----------+
|  0|         40|
|  0|         40|
|  0|         40|
|  0|         40|
|  0|         40|
+---+-----------+
only showing top 5 rows



## 1- splitting the columns into numeric and categorical

In [234]:
numeric_cols = [f for (f,d) in df_final.dtypes if ((d != 'string' ))]
categorical_cols = [f for (f,d) in df_final.dtypes if ((d=='string' ))]
print('numerical cols are ', numeric_cols)
print('categorical cols are ', categorical_cols)


numerical cols are  ['age', 'no_days', 'age_imputed']
categorical cols are  ['province', 'is_male', 'is_dead']


In [235]:
string_indexer_input = categorical_cols
string_indexer_output = [categorical_col + 'index' for categorical_col in categorical_cols]

ohe_input = string_indexer_output.copy()
ohe_input.remove('is_deadindex') # exclude the ouput (target variable)
ohe_output = [ohe_in + '_ohe' for ohe_in in ohe_input] 

assembler_input = ohe_output.copy()
assembler_input.extend( ['age_scaled' , 'no_days_scaled'] )
assembler_ouput = 'features'

print(ohe_input)
print(ohe_output)
print(assembler_input)

['provinceindex', 'is_maleindex']
['provinceindex_ohe', 'is_maleindex_ohe']
['provinceindex_ohe', 'is_maleindex_ohe', 'age_scaled', 'no_days_scaled']


## 2- train test split

In [236]:
from pyspark.ml.tuning import TrainValidationSplit, ParamGridBuilder
df_train , df_test = df_final.randomSplit([0.8,0.2] , seed=42)

In [237]:
df_final.columns

['age', 'province', 'no_days', 'is_male', 'is_dead', 'age_imputed']

## 3- normalizing the numeric columns

In [238]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml.linalg import Vectors
#Vectors.dense(df_final['age'])
age_vector_assembler = VectorAssembler(inputCols = ['age_imputed'] , outputCol='age_feature')
df_final_trial = age_vector_assembler.transform(df_train)
standard_scalar = StandardScaler(withMean=True) ## with mean true to subtract the mean
standard_scalar.setInputCol('age_feature')
standard_scalar.setOutputCol('age_scaled')
df_final_scaled = standard_scalar.fit(df_final_trial).transform(df_final_trial)
df_final_scaled.show(5,truncate=False)
df_final.select(fn.mean('age')).show()

+---+-----------------+-------+-------+-------+-----------+-----------+----------------------+
|age|province         |no_days|is_male|is_dead|age_imputed|age_feature|age_scaled            |
+---+-----------------+-------+-------+-------+-----------+-----------+----------------------+
|0  |Chungcheongbuk-do|49     |false  |false  |40         |[40.0]     |[-0.04159081026589833]|
|0  |Chungcheongnam-do|14     |false  |false  |40         |[40.0]     |[-0.04159081026589833]|
|0  |Chungcheongnam-do|16     |true   |false  |40         |[40.0]     |[-0.04159081026589833]|
|0  |Chungcheongnam-do|17     |true   |false  |40         |[40.0]     |[-0.04159081026589833]|
|0  |Chungcheongnam-do|17     |true   |false  |40         |[40.0]     |[-0.04159081026589833]|
+---+-----------------+-------+-------+-------+-----------+-----------+----------------------+
only showing top 5 rows

+------------------+
|          avg(age)|
+------------------+
|40.691131498470945|
+------------------+



normalize the number of days

In [239]:
no_days_vector_assembler = VectorAssembler(inputCols = ['no_days'] , outputCol='no_days_feature')
df_final_trial = no_days_vector_assembler.transform(df_final_scaled)
standard_scalar_2 = StandardScaler(withMean=True) ## with mean true to subtract the mean
standard_scalar_2.setInputCol('no_days_feature')
standard_scalar_2.setOutputCol('no_days_scaled')
df_final_scaled = standard_scalar_2.fit(df_final_trial).transform(df_final_trial)
df_final_scaled.show(5,truncate=False)
df_final.select(fn.mean('no_days')).show()

+---+-----------------+-------+-------+-------+-----------+-----------+----------------------+---------------+---------------------+
|age|province         |no_days|is_male|is_dead|age_imputed|age_feature|age_scaled            |no_days_feature|no_days_scaled       |
+---+-----------------+-------+-------+-------+-----------+-----------+----------------------+---------------+---------------------+
|0  |Chungcheongbuk-do|49     |false  |false  |40         |[40.0]     |[-0.04159081026589833]|[49.0]         |[1.8026269139597944] |
|0  |Chungcheongnam-do|14     |false  |false  |40         |[40.0]     |[-0.04159081026589833]|[14.0]         |[-0.771193589907975] |
|0  |Chungcheongnam-do|16     |true   |false  |40         |[40.0]     |[-0.04159081026589833]|[16.0]         |[-0.6241181325441025]|
|0  |Chungcheongnam-do|17     |true   |false  |40         |[40.0]     |[-0.04159081026589833]|[17.0]         |[-0.5505804038621662]|
|0  |Chungcheongnam-do|17     |true   |false  |40         |[40.0]    

In [240]:
df_final_scaled = df_final_scaled.drop('no_days' , 'age' , 'age_feature' , 'no_days_feature')
df_final_scaled.show(5,truncate = False)

+-----------------+-------+-------+-----------+----------------------+---------------------+
|province         |is_male|is_dead|age_imputed|age_scaled            |no_days_scaled       |
+-----------------+-------+-------+-----------+----------------------+---------------------+
|Chungcheongbuk-do|false  |false  |40         |[-0.04159081026589833]|[1.8026269139597944] |
|Chungcheongnam-do|false  |false  |40         |[-0.04159081026589833]|[-0.771193589907975] |
|Chungcheongnam-do|true   |false  |40         |[-0.04159081026589833]|[-0.6241181325441025]|
|Chungcheongnam-do|true   |false  |40         |[-0.04159081026589833]|[-0.5505804038621662]|
|Chungcheongnam-do|true   |false  |40         |[-0.04159081026589833]|[-0.5505804038621662]|
+-----------------+-------+-------+-----------+----------------------+---------------------+
only showing top 5 rows



## 3- string indexer (ordinal encoding)

In [241]:
string_indexer = StringIndexer(inputCols=string_indexer_input , 
                               outputCols=string_indexer_output ) 
                               #handleInvalid='skip') # no need to handle invalid as we removed the nulls
string_indexer_model = string_indexer.fit(df_final_scaled)
df_indexed = string_indexer_model.transform(df_final_scaled)
df_indexed.show(5)

+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+
|         province|is_male|is_dead|age_imputed|          age_scaled|      no_days_scaled|provinceindex|is_maleindex|is_deadindex|
+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+
|Chungcheongbuk-do|  false|  false|         40|[-0.0415908102658...|[1.8026269139597944]|          6.0|         0.0|         0.0|
|Chungcheongnam-do|  false|  false|         40|[-0.0415908102658...|[-0.771193589907975]|          1.0|         0.0|         0.0|
|Chungcheongnam-do|   true|  false|         40|[-0.0415908102658...|[-0.6241181325441...|          1.0|         1.0|         0.0|
|Chungcheongnam-do|   true|  false|         40|[-0.0415908102658...|[-0.5505804038621...|          1.0|         1.0|         0.0|
|Chungcheongnam-do|   true|  false|         40|[-0.0415908102658...|[-0.5505804038621...| 

## 4- One Hot Encoding

In [242]:
ohe = OneHotEncoder(inputCols=ohe_input, outputCols= ohe_output)
ohe_model = ohe.fit(df_indexed)
df_ohe = ohe_model.transform(df_indexed)
df_ohe.show(5)

+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+-----------------+----------------+
|         province|is_male|is_dead|age_imputed|          age_scaled|      no_days_scaled|provinceindex|is_maleindex|is_deadindex|provinceindex_ohe|is_maleindex_ohe|
+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+-----------------+----------------+
|Chungcheongbuk-do|  false|  false|         40|[-0.0415908102658...|[1.8026269139597944]|          6.0|         0.0|         0.0|   (15,[6],[1.0])|   (1,[0],[1.0])|
|Chungcheongnam-do|  false|  false|         40|[-0.0415908102658...|[-0.771193589907975]|          1.0|         0.0|         0.0|   (15,[1],[1.0])|   (1,[0],[1.0])|
|Chungcheongnam-do|   true|  false|         40|[-0.0415908102658...|[-0.6241181325441...|          1.0|         1.0|         0.0|   (15,[1],[1.0])|       (1,[],[])|
|Chungcheo

## 5- vector assembler

In [243]:
vector_assembler = VectorAssembler(inputCols = assembler_input , outputCol=assembler_ouput ) 
df_vector_out = vector_assembler.transform(df_ohe)
df_vector_out.show(5)


+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+-----------------+----------------+--------------------+
|         province|is_male|is_dead|age_imputed|          age_scaled|      no_days_scaled|provinceindex|is_maleindex|is_deadindex|provinceindex_ohe|is_maleindex_ohe|            features|
+-----------------+-------+-------+-----------+--------------------+--------------------+-------------+------------+------------+-----------------+----------------+--------------------+
|Chungcheongbuk-do|  false|  false|         40|[-0.0415908102658...|[1.8026269139597944]|          6.0|         0.0|         0.0|   (15,[6],[1.0])|   (1,[0],[1.0])|(18,[6,15,16,17],...|
|Chungcheongnam-do|  false|  false|         40|[-0.0415908102658...|[-0.771193589907975]|          1.0|         0.0|         0.0|   (15,[1],[1.0])|   (1,[0],[1.0])|(18,[1,15,16,17],...|
|Chungcheongnam-do|   true|  false|         40|[-0.0415908102658...|[-

In [244]:
df_model_in = df_vector_out.select('features' , 'is_deadindex')
df_model_in.show(5 , truncate = False)

+-------------------------------------------------------------------+------------+
|features                                                           |is_deadindex|
+-------------------------------------------------------------------+------------+
|(18,[6,15,16,17],[1.0,1.0,-0.04159081026589833,1.8026269139597944])|0.0         |
|(18,[1,15,16,17],[1.0,1.0,-0.04159081026589833,-0.771193589907975])|0.0         |
|(18,[1,16,17],[1.0,-0.04159081026589833,-0.6241181325441025])      |0.0         |
|(18,[1,16,17],[1.0,-0.04159081026589833,-0.5505804038621662])      |0.0         |
|(18,[1,16,17],[1.0,-0.04159081026589833,-0.5505804038621662])      |0.0         |
+-------------------------------------------------------------------+------------+
only showing top 5 rows



## 6- machine learning model

In [245]:
from pyspark.ml.classification import LogisticRegression , DecisionTreeClassifier , RandomForestClassifier 
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplit , ParamGridBuilder

lr = LogisticRegression(featuresCol='features' , labelCol='is_deadindex' , predictionCol='predictions')
grid = ParamGridBuilder().addGrid(lr.maxIter, [0, 1]).build()
evaluator = BinaryClassificationEvaluator(labelCol='is_deadindex' , rawPredictionCol='rawPrediction'  , metricName='areaUnderROC')

lr_model = lr.fit(df_vector_out)
df_predictions = lr_model.transform(df_model_in)
df_predictions.show(5,truncate=False)
evaluator.evaluate(df_predictions)

+-------------------------------------------------------------------+------------+----------------------------------------+-------------------------------------------+-----------+
|features                                                           |is_deadindex|rawPrediction                           |probability                                |predictions|
+-------------------------------------------------------------------+------------+----------------------------------------+-------------------------------------------+-----------+
|(18,[6,15,16,17],[1.0,1.0,-0.04159081026589833,1.8026269139597944])|0.0         |[40.732907122109715,-40.732907122109715]|[1.0,0.0]                                  |0.0        |
|(18,[1,15,16,17],[1.0,1.0,-0.04159081026589833,-0.771193589907975])|0.0         |[34.415496754256935,-34.415496754256935]|[0.9999999999999989,1.1102230246251565E-15]|0.0        |
|(18,[1,16,17],[1.0,-0.04159081026589833,-0.6241181325441025])      |0.0         |[33.3400956787021,

0.966455505279036

# making the pipe line

In [246]:
def evaluate_acc(results):
    
    results = results.select(['predictions', 'is_deadindex'])
    predictionAndLabels=results.rdd
    metrics = MulticlassMetrics(predictionAndLabels)

    cm=metrics.confusionMatrix().toArray()
    accuracy=(cm[0][0]+cm[1][1])/cm.sum()
    precision=(cm[0][0])/(cm[0][0]+cm[1][0])
    recall=(cm[0][0])/(cm[0][0]+cm[0][1])
    f_1 = 2*(precision*recall)/ (precision+recall)
    print("lr: accuracy,precision,recall , f1",accuracy,precision,recall , f_1)

In [247]:
stages = [age_vector_assembler , standard_scalar , no_days_vector_assembler ,standard_scalar_2\
         , string_indexer , ohe , vector_assembler , lr \
         ]

from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import BinaryClassificationMetrics , MulticlassMetrics

pipeline = Pipeline(stages = stages)
model = pipeline.fit(df_train)
df_test_predictions = model.transform(df_test)
evaluator.evaluate(df_test_predictions)
evaluate_acc(df_test_predictions)

lr: accuracy,precision,recall , f1 0.9612676056338029 0.9706959706959707 0.9888059701492538 0.9796672828096119
